In [2]:
import pandas as pd
import numpy as np

from datetime import datetime
from merlion.utils.time_series import TimeSeries as merlion_timeSeries
from omnixai.data.timeseries import Timeseries as omnixai_timeSeries

from merlion.models.defaults import DefaultForecasterConfig, DefaultForecaster
from omnixai.explainers.timeseries.agnostic.shap import ShapTimeseries

In [3]:
#Read Data
url='https://raw.githubusercontent.com/AmirMK/XAI_Time_Seris_SF/main/my_test_data.csv'
df = pd.read_csv(url)
time = df['Month of Datetime'].tolist()
target = df['#Passengers'].tolist() 

#Dara Pre-Processing
time = [datetime.strptime(date[:10],'%Y-%m-%d').date() for date in time]
df = pd.DataFrame(
        {'Datetime': time,
         'Target': target
        })
df['Datetime'] = pd.to_datetime(df['Datetime'])
df = df.set_index('Datetime')
df.index = pd.to_datetime(df.index)
df.index.freq = 'MS'

In [4]:
#Train data with merlion_timeSeries as merlion is used to build the model
train_data = df[~df['Target'].isna()]
train_data = merlion_timeSeries.from_pd(train_data)

#Test data with omnixai_timeSeries as omnixai is used to explain the model
test_data = df[df['Target'].isna()]
test_data['Target']=1
test_data = omnixai_timeSeries.from_pd(test_data)


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [5]:
#Creat and train the model
model_config = DefaultForecasterConfig()
model = DefaultForecaster(model_config)

train = model.train(train_data)

In [6]:
#forecasting function accept omnixai_timeSeries

def forecasting_function(time_stamps):
    #convert omnixai_timeSeries to merlion_timeSeries 
    df = time_stamps.to_pd()
    _input=merlion_timeSeries.from_pd(df)   
    #Get the forecats and generate output as numpy array
    output = model.forecast(_input.time_stamps)
    r = output[0].to_pd().reset_index()['Target'].tolist()
    return np.array(r)

In [7]:
#test the forecasting function
forecasting_function(test_data)

array([451.35407509, 426.72170582, 464.55734934, 500.06614872,
       508.5295283 , 567.39469818, 647.06695076, 631.13057054,
       532.50336324, 482.29419464, 415.24519801, 459.42924934,
       478.23392554, 453.06395926, 490.37275771, 525.36524892,
       533.32264649, 591.69195402, 670.87826147, 654.46565505,
       555.37174606, 504.7052098 , 437.20799287, 480.9527883 ,
       499.32699371, 473.73516607, 510.63054039, 545.21787595,
       552.77822098, 610.75841701, 689.56339521, 672.77708611,
       573.31694849, 522.29150819, 454.44256529, 497.84266927,
       515.87907707, 489.95620776, 526.52716124, 560.79656438,
       568.04533565, 625.72018939, 704.22593214, 687.1463723 ,
       587.39884896, 536.09177064, 467.9668225 , 511.09644133])

In [8]:
#Creat shap explnaier 
explainers = ShapTimeseries(
    training_data=omnixai_timeSeries.from_pd(train_data.to_pd()), #train data as merlion_timeSeries converted to omnixai_timeSeries 
    predict_function =forecasting_function,
    mode ='forecasting'
)

In [9]:
explanations = explainers.explain(test_data)

Provided model function fails when applied to the provided data set.


ValueError: Must pass 2-d input. shape=(96, 48, 1)